In [1]:
import os
import pandas as pd
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis

## Paring down the iJN1463 to a smaller scale model

#### Summary of changes:

* Removed `pWW0_xxx` genes from the model because `TOL` plasmid is absent in *P. putida* KT2440
* Removed or fixed mass or charge imbalanced reactions
   1. Five reactions were removed since correct balanced reactions are in the model: `MEPCT_1, HDH, APPAT, ARGDI, IDPh_1`
   2. Modified two reactions to correct the mass or charge balance
      `FE3PYOVDDR (fe2_p -> fe3_p), PPRGL (2.0 h_c -> h_c)`
   3. Modified four reactions and six metabolites to correct inconsistent reactions in the pyrroloquinoline quinone biosynthesis
      `PQQAC, PQQFEP, 2PQQS, PQQH2S`
      `pqqA_kt_c, pqqAc_kt_c, 4pqq_c, 3pqq_c, 2pqq_c, 1pqq_c`
* Removed sub-reactions for pyruvate dehydrogenase and alpha-ketoglutarate dehydrogenase
  `PDHa, PDHbr, AKGDa, AKGDb_copy1, AKGDb_copy2`
  The reaction for alpha-ketoglutarate dehydrogenase is now irreversible


### Load the model from Bigg    

http://bigg.ucsd.edu/static/models/iJN1463.json

In [2]:
# Load the model from BiGG:  
model = cobra.io.load_json_model('../models/iJN1463.json')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-23


In [3]:
# Change the solver to cplex to avoid renaming issues in gurobi optlang interface
#model.solver = 'cplex'
model.tolerance = 1e-9

In [4]:
# Check biomass reactions
for r in model.reactions:
    if 'BIOMASS' in r.id:
        print(r.id, r.bounds)

BIOMASS_KT2440_Core2 (0.0, 999999.0)
BIOMASS_KT2440_WT3 (0.0, 999999.0)


In [5]:
# Remove the irrelevent biomass equation
model.reactions.BIOMASS_KT2440_Core2.upper_bound = 0.0

#### Remove pWW0_xxx genes on the TOL plasmid

In [6]:
cobra.manipulation.remove_genes(model, [g for g in model.genes if not g.id.startswith('PP_')])

#### Fix unbalanced reactions

In [7]:
for r in model.reactions:
    if not r.boundary and not 'BIOMASS' in r.id and r.check_mass_balance():
        print(r, r.gene_reaction_rule)
        print(r.check_mass_balance())
        for m in r.metabolites:
            print(m.id, m.formula, m.charge)
        print()

FE3PYOVDDR: fe3pyovd_p <=> fe2_p + pyovd_p 
{'charge': -1.0}
fe2_p Fe 2
fe3pyovd_p RFe 3
pyovd_p R 0

PQQFEP: 4.0 h2o_c + pqqAc_kt_c --> 4pqq_c + tripeptide_c PP_0381
{'H': 1.0}
4pqq_c C14H15N2O7 -3
h2o_c H2O 0
pqqAc_kt_c RC14H6N2O3 -2
tripeptide_c R 1

MEPCT_1: 2me4p_c + ctp_c --> 4c2me_c + ppi_c PP_1614
{'charge': 1.0, 'H': 1.0}
2me4p_c C5H11O7P -2
4c2me_c C14H23N3O14P2 -2
ctp_c C9H12N3O14P3 -4
ppi_c HO7P2 -3

HDH: h2o_c + histd_c + 2.0 nad_c --> 4.0 h_c + his__L_c + 2.0 nadh_c PP_0966
{'charge': 1.0, 'H': 1.0}
h2o_c H2O 0
h_c H 1
his__L_c C6H9N3O2 0
histd_c C6H12N3O 1
nad_c C21H26N7O14P2 -1
nadh_c C21H27N7O14P2 -2

APPAT: atp_c + 2.0 h_c + pan4p_c --> dpcoa_c + ppi_c PP_5123
{'charge': -1.0, 'H': -1.0}
atp_c C10H12N5O13P3 -4
dpcoa_c C21H33N7O13P2S -2
h_c H 1
pan4p_c C11H21N2O7PS -2
ppi_c HO7P2 -3

ARGDI: arg__L_c + h2o_c + h_c --> citr__L_c + nh4_c PP_1001
{'charge': -1.0, 'H': -1.0}
arg__L_c C6H15N4O2 1
citr__L_c C6H13N3O3 0
h2o_c H2O 0
h_c H 1
nh4_c H4N 1

IDPh_1: h2o_c + ppi_c --

In [8]:
for r in model.reactions:
    if not r.boundary and not 'BIOMASS' in r.id and r.check_mass_balance():
        print(r, r.gene_reaction_rule)
        for g in r.genes:
            print(g.id)
            for r2 in g.reactions:
                print(r2, r2.check_mass_balance())
        print()

FE3PYOVDDR: fe3pyovd_p <=> fe2_p + pyovd_p 

PQQFEP: 4.0 h2o_c + pqqAc_kt_c --> 4pqq_c + tripeptide_c PP_0381
PP_0381
PQQFEP: 4.0 h2o_c + pqqAc_kt_c --> 4pqq_c + tripeptide_c {'H': 1.0}

MEPCT_1: 2me4p_c + ctp_c --> 4c2me_c + ppi_c PP_1614
PP_1614
MEPCT_1: 2me4p_c + ctp_c --> 4c2me_c + ppi_c {'charge': 1.0, 'H': 1.0}
MEPCT: 2me4p_c + ctp_c + h_c --> 4c2me_c + ppi_c {}

HDH: h2o_c + histd_c + 2.0 nad_c --> 4.0 h_c + his__L_c + 2.0 nadh_c PP_0966
PP_0966
HDH: h2o_c + histd_c + 2.0 nad_c --> 4.0 h_c + his__L_c + 2.0 nadh_c {'charge': 1.0, 'H': 1.0}
HISTD: h2o_c + histd_c + 2.0 nad_c --> 3.0 h_c + his__L_c + 2.0 nadh_c {}

APPAT: atp_c + 2.0 h_c + pan4p_c --> dpcoa_c + ppi_c PP_5123
PP_5123
PTPATi: atp_c + h_c + pan4p_c --> dpcoa_c + ppi_c {}
APPAT: atp_c + 2.0 h_c + pan4p_c --> dpcoa_c + ppi_c {'charge': -1.0, 'H': -1.0}

ARGDI: arg__L_c + h2o_c + h_c --> citr__L_c + nh4_c PP_1001
PP_1001
ARGDr: arg__L_c + h2o_c --> citr__L_c + nh4_c {}
ARGDI: arg__L_c + h2o_c + h_c --> citr__L_c + nh4_c 

In [9]:
model.remove_reactions(['MEPCT_1', 'HDH', 'APPAT', 'ARGDI', 'IDPh_1'], remove_orphans=True)
model.reactions.FE3PYOVDDR.add_metabolites({'fe2_p': -1.0, 'fe3_p': 1.0})
model.reactions.PPRGL.add_metabolites({'h_c': -1.0})
model.metabolites.rephac_c.formula = 'C8H8O3'

In [10]:
for r in model.reactions:
    if not r.boundary and not 'BIOMASS' in r.id and r.check_mass_balance():
        print(r, r.gene_reaction_rule)
        print(r.check_mass_balance())
        for m in r.metabolites:
            print(m.id, m.formula, m.charge)
        print()

PQQFEP: 4.0 h2o_c + pqqAc_kt_c --> 4pqq_c + tripeptide_c PP_0381
{'H': 1.0}
4pqq_c C14H15N2O7 -3
h2o_c H2O 0
pqqAc_kt_c RC14H6N2O3 -2
tripeptide_c R 1

BVMO5: h_c + nadph_c + o2_c + und2one_c --> h2o_c + 0.22 methdca_c + nadp_c + 0.78 nnac_c PP_2805
{'O': 1.3322676295501878e-15, 'C': 1.7763568394002505e-15}
h2o_c H2O 0
h_c H 1
methdca_c C11H22O2 0
nadp_c C21H25N7O17P3 -3
nadph_c C21H26N7O17P3 -4
nnac_c C11H22O2 0
o2_c O2 0
und2one_c C11H22O 0



PQQFEP in pyrroloquinoline quinone biosynthesis, incorrect reactions  
see https://biocyc.org/META/NEW-IMAGE?type=PATHWAY&object=PWY-6420&detail-level=3  
BVMO5 balanced  

In [11]:
model.metabolites.get_by_id('pqqA_kt_c').formula = 'RC14H11N2O3'
model.metabolites.get_by_id('pqqA_kt_c').charge = 0
model.metabolites.get_by_id('pqqAc_kt_c').formula = 'RC14H9N2O3'
model.metabolites.get_by_id('pqqAc_kt_c').charge = 0
model.reactions.get_by_id('PQQAC').add_metabolites({'amet_c': -1.0, 'met__L_c': 1.0, 'dad_5_c': 1.0, 'h_c': -1.0})
model.reactions.get_by_id('PQQFEP').gene_reaction_rule = 'PP_0380 and PP_0381'
model.metabolites.get_by_id('4pqq_c').formula = 'C14H17N2O7'
model.metabolites.get_by_id('4pqq_c').charge = -1
model.metabolites.get_by_id('3pqq_c').formula = 'C14H17N2O7'
model.metabolites.get_by_id('3pqq_c').charge = -1
model.reactions.get_by_id('2PQQS').add_metabolites({'h_c': 1.0})
model.metabolites.get_by_id('2pqq_c').formula = 'C14H14N2O6'
model.metabolites.get_by_id('2pqq_c').charge = -2
model.metabolites.get_by_id('1pqq_c').formula = 'C14H14N2O8'
model.metabolites.get_by_id('1pqq_c').charge = -2
model.reactions.get_by_id('PQQH2S').add_metabolites({'h_c': 1.0})

In [12]:
for r in model.reactions:
    if not r.boundary and not 'BIOMASS' in r.id and r.check_mass_balance():
        print(r, r.gene_reaction_rule)
        print(r.check_mass_balance())
        for m in r.metabolites:
            print(m.id, m.formula, m.charge)
        print()

BVMO5: h_c + nadph_c + o2_c + und2one_c --> h2o_c + 0.22 methdca_c + nadp_c + 0.78 nnac_c PP_2805
{'O': 1.3322676295501878e-15, 'C': 1.7763568394002505e-15}
h2o_c H2O 0
h_c H 1
methdca_c C11H22O2 0
nadp_c C21H25N7O17P3 -3
nadph_c C21H26N7O17P3 -4
nnac_c C11H22O2 0
o2_c O2 0
und2one_c C11H22O 0



#### Check duplicate reactions

In [13]:
for r in sorted(model.reactions, key=lambda x:x.id):
    if 'copy' in r.id:
        print(r, r.gene_reaction_rule)

AKGDb_copy1: coa_c + sdhlam_c <=> dhlam_c + succoa_c PP_4188
AKGDb_copy2: coa_c + sdhlam_c --> dhlam_c + succoa_c PP_4188
ALDD3y_copy1: h2o_c + nadp_c + ppal_c --> 2.0 h_c + nadph_c + ppa_c 
ALDD3y_copy2: h2o_c + nadp_c + ppal_c --> 2.0 h_c + nadph_c + ppa_c PP_2492
DHAD1_copy1: 23dhmb_c --> 3mob_c + h2o_c PP_5128
DHAD1_copy2: 23dhmb_c --> 3mob_c + h2o_c PP_5128
DHQTi_copy1: 3dhq_c <=> 3dhsk_c + h2o_c PP_0560 or PP_3003 or PP_2407
DHQTi_copy2: 3dhq_c --> 3dhsk_c + h2o_c PP_0560 or PP_2407 or PP_3003
FMNRx2_copy1: fmn_c + h_c + nadph_c <=> fmnh2_c + nadp_c PP_1638
FMNRx2_copy2: fmn_c + h_c + nadph_c --> fmnh2_c + nadp_c PP_0236 or PP_1638 or PP_4646
G5SD_copy1: glu5p_c + h_c + nadph_c --> glu5sa_c + nadp_c + pi_c PP_4811
G5SD_copy2: glu5p_c + h_c + nadph_c <=> glu5sa_c + nadp_c + pi_c PP_4811
GLNS_copy1: atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c PP_5046
GLNS_copy2: atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c PP_5046
GLUCYS_copy1: atp_c + cys__L_c + glu__L

Keep them for now, it would not make a difference in model predictions

#### Remove duplicate reactions for PDH/AKGDH/GLYCL

In [14]:
for x in ['PP_0338','PP_0339','PP_0986','PP_0988','PP_0989','PP_4187','PP_4188','PP_4189',
          'PP_5192','PP_5193','PP_5194','PP_5366']:
    for r in model.genes.get_by_id(x).reactions:
        print(r, r.gene_reaction_rule)
    print()

PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c (PP_0339 and PP_0338 and PP_4187) or (PP_0339 and PP_0338 and PP_5366)
PDHbr: adhlam_c + coa_c <=> accoa_c + dhlam_c PP_0338

PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c (PP_0339 and PP_0338 and PP_4187) or (PP_0339 and PP_0338 and PP_5366)
PDHa: h_c + lpam_c + pyr_c --> adhlam_c + co2_c PP_0339

FOMETRi: 5fthf_c + h_c --> h2o_c + methf_c PP_0986 or PP_5194
GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c (PP_5193 and PP_5192 and PP_5194 and PP_4187) or (PP_0989 and PP_0988 and PP_0986 and PP_4187)

GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c (PP_5193 and PP_5192 and PP_5194 and PP_4187) or (PP_0989 and PP_0988 and PP_0986 and PP_4187)

GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c (PP_5193 and PP_5192 and PP_5194 and PP_4187) or (PP_0989 and PP_0988 and PP_0986 and PP_4187)

AKGDH: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c PP_4189 and PP_4188 and PP_

In [15]:
with model:
    sol = model.optimize()
    print(sol.objective_value)

    # compare whether removing reactions affects the FBA results
    temp = ['AKGDa','AKGDb_copy1','AKGDb_copy2','PDHa','PDHbr']
    model.remove_reactions(temp, remove_orphans=True)
    model.reactions.PDHcr.lower_bound = 0.0
    sol = model.optimize()
    print(sol.objective_value)    

0.5838113260876141
0.5838113260876154


In [16]:
for x in ['PP_0338','PP_0339','PP_0986','PP_0988','PP_0989','PP_4187','PP_4188','PP_4189',
          'PP_5192','PP_5193','PP_5194','PP_5366']:
    for r in model.genes.get_by_id(x).reactions:
        print(r, r.gene_reaction_rule)
    print()

PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c (PP_0339 and PP_0338 and PP_4187) or (PP_0339 and PP_0338 and PP_5366)
PDHbr: adhlam_c + coa_c <=> accoa_c + dhlam_c PP_0338

PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c (PP_0339 and PP_0338 and PP_4187) or (PP_0339 and PP_0338 and PP_5366)
PDHa: h_c + lpam_c + pyr_c --> adhlam_c + co2_c PP_0339

FOMETRi: 5fthf_c + h_c --> h2o_c + methf_c PP_0986 or PP_5194
GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c (PP_5193 and PP_5192 and PP_5194 and PP_4187) or (PP_0989 and PP_0988 and PP_0986 and PP_4187)

GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c (PP_5193 and PP_5192 and PP_5194 and PP_4187) or (PP_0989 and PP_0988 and PP_0986 and PP_4187)

GLYCL: gly_c + nad_c + thf_c --> co2_c + mlthf_c + nadh_c + nh4_c (PP_5193 and PP_5192 and PP_5194 and PP_4187) or (PP_0989 and PP_0988 and PP_0986 and PP_4187)

AKGDH: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c PP_4189 and PP_4188 and PP_

#### Check the boundary condition

In [17]:
model.medium

{'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_co2_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_glc__D_e': 6.0,
 'EX_h2o_e': 100.0,
 'EX_h_e': 100.0,
 'EX_hco3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_nh4_e': 30.0,
 'EX_ni2_e': 10.0,
 'EX_o2_e': 100.0,
 'EX_pi_e': 10.0,
 'EX_sel_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_tungs_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_acmtsoxin_e': 1000.0,
 'EX_acpptrn_e': 1000.0,
 'EX_d2one_e': 1000.0,
 'EX_d3one_e': 1000.0,
 'EX_d4one_e': 1000.0,
 'EX_mtsoxin_e': 1000.0,
 'EX_n2one_e': 1000.0,
 'EX_pptrn_e': 1000.0,
 'EX_und2one_e': 1000.0}

In [18]:
for x in ['EX_acmtsoxin_e', 'EX_acpptrn_e', 'EX_d2one_e', 'EX_d3one_e', 'EX_d4one_e', 'EX_mtsoxin_e',
          'EX_n2one_e', 'EX_pptrn_e', 'EX_und2one_e']:
    r = model.reactions.get_by_id(x)
    print(r, r.name)

EX_acmtsoxin_e: acmtsoxin_e <=>  N Acetylmethionine sulfoximine exchange
EX_acpptrn_e: acpptrn_e <=>  N Acetylphosphinothricin exchange
EX_d2one_e: d2one_e <=>  2 Decanone exchange
EX_d3one_e: d3one_e <=>  3 Decanone exchange
EX_d4one_e: d4one_e <=>  4 Decanone exchange
EX_mtsoxin_e: mtsoxin_e <=>  Methionine sulfoximine exchange
EX_n2one_e: n2one_e <=>  2 Nonanone exchange
EX_pptrn_e: pptrn_e <=>  L Phosphinothricin exchange
EX_und2one_e: und2one_e <=>  2 Undecanone exchange


In [19]:
with model:
    sol = model.optimize()
    print(sol.objective_value)
    
    # compare whether removing reactions affects the FBA results
    for x in ['EX_acmtsoxin_e', 'EX_acpptrn_e', 'EX_d2one_e', 'EX_d3one_e', 'EX_d4one_e', 'EX_mtsoxin_e',
              'EX_n2one_e', 'EX_pptrn_e', 'EX_und2one_e']:
        model.reactions.get_by_id(x).lower_bound = 0.0
    sol = model.optimize()
    print(sol.objective_value)

0.5838113260876157
0.5838113260876154


In [20]:
for x in ['EX_acmtsoxin_e', 'EX_acpptrn_e', 'EX_d2one_e', 'EX_d3one_e', 'EX_d4one_e', 'EX_mtsoxin_e',
          'EX_n2one_e', 'EX_pptrn_e', 'EX_und2one_e']:
    model.reactions.get_by_id(x).lower_bound = 0.0

#### Check other reactions with non-zero bounds

In [21]:
for r in model.reactions:
    if r.lower_bound > -1000.0 and r.lower_bound:
        print(r, r.bounds)
print()
for r in model.reactions:
    if r.upper_bound < 1000.0 and r.upper_bound:
        print(r, r.bounds)

EX_ca2_e: ca2_e <=>  (-10.0, 999999.0)
EX_cl_e: cl_e <=>  (-10.0, 999999.0)
EX_co2_e: co2_e <=>  (-100.0, 999999.0)
EX_cobalt2_e: cobalt2_e <=>  (-10.0, 999999.0)
EX_cu2_e: cu2_e <=>  (-10.0, 999999.0)
EX_fe2_e: fe2_e <=>  (-10.0, 999999.0)
EX_glc__D_e: glc__D_e <=>  (-6.0, 1000.0)
EX_h2o_e: h2o_e <=>  (-100.0, 999999.0)
EX_h_e: h_e <=>  (-100.0, 999999.0)
EX_hco3_e: hco3_e <=>  (-10.0, 999999.0)
EX_k_e: k_e <=>  (-10.0, 999999.0)
EX_mg2_e: mg2_e <=>  (-10.0, 999999.0)
EX_mn2_e: mn2_e <=>  (-10.0, 999999.0)
EX_mobd_e: mobd_e <=>  (-10.0, 999999.0)
EX_na1_e: na1_e <=>  (-10.0, 999999.0)
EX_nh4_e: nh4_e <=>  (-30.0, 1000.0)
EX_ni2_e: ni2_e <=>  (-10.0, 999999.0)
EX_o2_e: o2_e <--  (-100.0, 0.0)
EX_pi_e: pi_e <=>  (-10.0, 999999.0)
EX_sel_e: sel_e <=>  (-10.0, 999999.0)
EX_so4_e: so4_e <=>  (-10.0, 999999.0)
EX_tungs_e: tungs_e <=>  (-10.0, 999999.0)
EX_zn2_e: zn2_e <=>  (-10.0, 999999.0)
SK_pqqA_kt_c: pqqA_kt_c <--  (-1.0, 0.0)
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c (0.92, 0.92)

ATP

#### Replace arbitrary bounds with -1000.0 or 1000.0

In [22]:
with model:
    sol = model.optimize()
    print(sol.objective_value)
    for r in model.reactions:
        if r.lower_bound < -1000.0 and r.lower_bound and r.id not in ['ATPM','EX_glc__D_e']:
            r.lower_bound = -1000.0
    sol = model.optimize()
    print(sol.objective_value)

0.5838113260876154
0.5838113260876154


In [23]:
for r in model.reactions:
    if r.lower_bound < -1000.0 and r.lower_bound and r.id not in ['ATPM','EX_glc__D_e']:
        r.lower_bound = -1000.0

In [24]:
for r in model.reactions:
    if r.lower_bound <= -1000.0:
        r.lower_bound = -1000.0
    if r.upper_bound >= 1000.0:
        r.upper_bound = 1000.0

In [25]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.002467,0,0.00%
cl_e,EX_cl_e,0.002467,0,0.00%
cobalt2_e,EX_cobalt2_e,0.001775,0,0.00%
cu2_e,EX_cu2_e,0.001645,0,0.00%
fe2_e,EX_fe2_e,0.008574,0,0.00%
glc__D_e,EX_glc__D_e,6,6,99.99%
k_e,EX_k_e,0.09254,0,0.00%
mg2_e,EX_mg2_e,0.004112,0,0.00%
mn2_e,EX_mn2_e,0.001645,0,0.00%
mobd_e,EX_mobd_e,0.001905,0,0.00%


#### Remove PPCK; PP_0253 is annotated as a pseudogene

`PP_0253` is annotated as a pseudogene due to a frameshift mutation that results in the loss of function. 

Copied below is the summary from 

Belda, E., van Heck, R. G. A., José Lopez-Sanchez, M., Cruveiller, S., Barbe, V., Fraser, C., Klenk, H.-P., Petersen, J., Morgat, A., Nikel, P. I., Vallenet, D., Rouy, Z., Sekowska, A., Martins Dos Santos, V. A. P., de Lorenzo, V., Danchin, A., & Médigue, C. (2016). The revisited genome of Pseudomonas putida KT2440 enlightens its value as a robust metabolic chassis. Environmental Microbiology, 18(10), 3403–3424. https://doi.org/10.1111/1462-2920.13230

Joonhoon

```
The gene PP_0253 is split into two fragments that have 100% amino acid identity with fragments of the pckA gene encoding phosphoenolpyruvate carboxykinase (ATP dependent) in P. putida F1 (UniProt entry A5VX32). This enzyme is involved in gluconeogenesis, where it catalyzes the conversion of oxaloacetate (OAA) to phosphoenolpyruvate (PEP). The present UniProt functional annotation is supported by sequence similarity using the UniRule annotation procedure (The UniProt Consortium, 2014). Indeed, similarity with an experimentally validated phosphoenolpyruvate carboxykinase is found with the Staphylococcus aureus PckA protein (Q2G1W2, 45.4% amino-acid identity) (Scovill et al., 1996). The underlying reason for this loss of function in strain KT2440 is unknown, but we note that this enzyme is a key enzyme required for gluconeogenesis, under conditions where P. putida strains display a tight regulation of the balance between fluxes going from glucose to pyruvate and from succinate to pyruvate (La Rosa et al., 2015). In Escherichia coli O157:H7, PckA is important for maintaining the pathogenic bacteria in competition with the bulk of the microbiota (Bertin et al., 2014); inactivation of the gene may contribute to the GRAS phenotype of strain KT2440. Additionally, the enzyme is allosterically regulated by Ca21 in other g-proteobacteria (Sudom et al., 2003), and this feature might point at a particular role of the inactivation of this gene in the P. putida KT2440 niche. 
```


#### Update GPR for OAADC from PP_1024 (eda) to PP_1389 (Oxaloacetate decarboxylase)

In [26]:
model.remove_reactions(['PPCK'], remove_orphans=True)
model.reactions.OAADC.gene_reaction_rule = 'PP_1389'
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.002467,0,0.00%
cl_e,EX_cl_e,0.002467,0,0.00%
cobalt2_e,EX_cobalt2_e,0.001775,0,0.00%
cu2_e,EX_cu2_e,0.001645,0,0.00%
fe2_e,EX_fe2_e,0.008574,0,0.00%
glc__D_e,EX_glc__D_e,6,6,99.99%
k_e,EX_k_e,0.09254,0,0.00%
mg2_e,EX_mg2_e,0.004112,0,0.00%
mn2_e,EX_mn2_e,0.001645,0,0.00%
mobd_e,EX_mobd_e,0.001905,0,0.00%


# Savepoint


In [27]:
model = cobra.io.load_json_model('../models/iJN1463_savepoint_JS.json')
model.tolerance = 1e-9

## Add in 4ACA pathway reactions

In [28]:
M_4a4dpr_c = Metabolite('4a4dpr_c',
    formula='C10H10NO5',
    name='4-amino-4-deoxyprephenate',
    charge=-1,
    compartment='c')

M_4aPPA_c = Metabolite('4aPPA_c',
    formula='C9H8NO3',
    name='3-(4-aminophenyl)pyruvate',
    charge=-1,
    compartment='c')

M_4aPhe_c = Metabolite('4aPhe_c',
    formula='C9H12N2O2',
    name='4-L-aminophenylalanine',
    charge=0,                   
    compartment='c')

M_4aca_c = Metabolite('4aca_c',
    formula='C9H9NO2',
    name='4-aminocinnamic acid',
    charge=0,  
    compartment='c')

M_4asty_c = Metabolite('4asty_c',
    formula='C8H9N',
    name='4-aminostyrene',
    charge=0,  
    compartment='c')

model.add_metabolites([M_4a4dpr_c, M_4aPPA_c, M_4aPhe_c, M_4aca_c, M_4asty_c])

In [29]:
reaction = Reaction('R_PABASYN_RXN')
reaction.name = 'EC 2.6.1.85'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.gene_reaction_rule = '( papA )'
model.add_reactions([reaction])
model.reactions.R_PABASYN_RXN.add_metabolites({'chor_c': -1.0, 'gln__L_c': -1.0, '4adcho_c':1.0, 'glu__L_c':1.0})

reaction = Reaction('RXN_20192')
reaction.name = 'EC 5.4.99.67'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.gene_reaction_rule = '( papB )'
model.add_reactions([reaction])
model.reactions.RXN_20192.add_metabolites({'4adcho_c': -1.0, '4a4dpr_c':1.0})

reaction = Reaction('RXN_20193')
reaction.name = 'EC 1.3.1.121'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.gene_reaction_rule = '( papC )'
model.add_reactions([reaction])
model.reactions.RXN_20193.add_metabolites({'4a4dpr_c': -1.0, 'nad_c':-1.0, '4aPPA_c':1.0, 'co2_c':1.0, 'nadh_c':1.0, 'h_c':1.0})

reaction = Reaction('RXN_20194')
reaction.name = 'EC 2.6.1.'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
model.reactions.RXN_20194.add_metabolites({'4aPhe_c': -1.0, 'akg_c':-1.0, '4aPPA_c':1.0, 'glu__L_c':1.0})

reaction = Reaction('RgPAL')
reaction.name = 'EC 4.3.1.24/4.3.1.25'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.gene_reaction_rule = '( Rg-PAL )'
model.add_reactions([reaction])
model.reactions.RgPAL.add_metabolites({'4aPhe_c': -1.0, 'h_c':-1.0, '4aca_c':1.0, 'nh4_c':1.0})

In [30]:
M_4aca_e = Metabolite('4aca_e',
    formula='C9H9NO2',
    name='4-aminocinnamic acid',
    charge=0,  
    compartment='e')

model.add_metabolites([M_4aca_e])

# add in internal 4-amminocinnamate --> external 4-amminocinnamate ) 
reaction = Reaction('R_4aca_Tr')
reaction.name = '4aca_c_TRANSPORT'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
model.reactions.R_4aca_Tr.add_metabolites({'4aca_c': -1.0, '4aca_e': 1.0})

model.add_boundary(model.metabolites.get_by_id("4aca_e"), type="exchange")

Reaction identifier,EX_4aca_e
Name,4-aminocinnamic acid exchange
Memory address,0x1a5ecef8eb0
Stoichiometry,4aca_e <=> 4-aminocinnamic acid <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [31]:
a = Metabolite('4abz_e',
    formula='C7H6NO2',
    name='4-aminobenzoate',
    charge=-1,
    compartment='e')

b = Metabolite('4aPhe_e',
    formula='C9H12N2O2',
    name='4-L-aminophenylalanine',
    charge=0,
    compartment='e')

model.add_metabolites([a, b])

reaction = Reaction('TRANS-RXN18UU-18')
reaction.name = 'TRANS-RXN18UU-18'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.gene_reaction_rule = '( PP_5297 )'
reaction.add_metabolites({'4abz_c': -1.0, '4abz_e': 1.0})

reaction = Reaction('TRANS-RXN18UU-62')
reaction.name = 'TRANS-RXN18UU-62'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.gene_reaction_rule = '( PP_3658 )'
reaction.add_metabolites({'4aPhe_c': -1.0, '4aPhe_e': 1.0})

model.add_boundary(model.metabolites.get_by_id("4abz_e"), type="exchange")
model.add_boundary(model.metabolites.get_by_id("4aPhe_e"), type="exchange")

Reaction identifier,EX_4aPhe_e
Name,4-L-aminophenylalanine exchange
Memory address,0x1a5873479d0
Stoichiometry,4aPhe_e <=> 4-L-aminophenylalanine <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [32]:
M_phpyr = Metabolite('phpyr_c',
    formula='C9H7O3',
    name='2-oxo-3-phenylpropanoate',
    charge=-1,
    compartment='c')

M_cinm_c = Metabolite('cinm_c',
    formula='C9H7O2',
    name='cinnamic acid',
    charge=-1,
    compartment='c')

M_cinm_e = Metabolite('cinm_e',
    formula='C9H7O2',
    name='cinnamic acid',
    charge=-1,
    compartment='e')

M_nh4 = Metabolite('nh4_c',
    formula='H4N',
    name='ammonia',
    charge=1,
    compartment='c')
    
model.add_metabolites([M_phpyr, M_cinm_c, M_cinm_e, M_nh4])


In [33]:
reaction = Reaction('R_pheA')
reaction.name = '4.2.1.51'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.gene_reaction_rule = '( PP_1769 )'
reaction.add_metabolites({'pphn_c':-1.0, 'h_c':-1.0,  'phpyr_c':1.0, 'co2_c':1.0, 'h2o_c':1.0})

reaction = Reaction('PAL_F')
reaction.name = 'EC 4.3.1.24/4.3.1.25'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({'phe__L_c':-1.0,  'cinm_c':1.0, 'nh4_c':1.0})

reaction = Reaction('R_cinm_Tr')
reaction.name = 'cinm_TRANSPORT'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({'cinm_c':-1.0, 'cinm_e':1.0})

model.add_boundary(model.metabolites.get_by_id("cinm_e"), type="exchange")

Reaction identifier,EX_cinm_e
Name,cinnamic acid exchange
Memory address,0x1a5ec0c99a0
Stoichiometry,cinm_e <=> cinnamic acid <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [34]:
# another name for p-coumaric acid
M_pHCA_c = Metabolite('pHCA_c',
    formula='C9H7O3',
    name='4-hydroxycinnamate',
    charge=-1,
    compartment='c')

M_pHCA_e = Metabolite('pHCA_e',
    formula='C9H7O3',
    name='4-hydroxycinnamate',
    charge=-1,
    compartment='e')

model.add_metabolites([M_pHCA_c, M_pHCA_e])

In [35]:
reaction = Reaction('R_tyrA')
reaction.name = 'E. coli tyrA'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.gene_reaction_rule = '( PP_1770 )'
reaction.add_metabolites({'pphn_c':-1.0, 'nad_c':-1.0,  '34hpp_c':1.0, 'co2_c':1.0, 'nadh_c':1.0})

reaction = Reaction('PAL_Y')
reaction.name = '2.6.1.57 Y'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.gene_reaction_rule = '( PP_4692 )'
reaction.add_metabolites({'tyr__L_c':-1.0, 'pHCA_c':1.0, 'nh4_c':1.0})

reaction = Reaction('R_pHCA_Tr')
reaction.name = 'pHCA_TRANSPORT'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({'pHCA_c':-1.0, 'pHCA_e':1.0})

model.add_boundary(model.metabolites.get_by_id("pHCA_e"), type="exchange")

Reaction identifier,EX_pHCA_e
Name,4-hydroxycinnamate exchange
Memory address,0x1a5ed56b9d0
Stoichiometry,pHCA_e <=> 4-hydroxycinnamate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [36]:
model.reactions.EX_4aca_e.lower_bound = 0
model.reactions.EX_cinm_e.lower_bound = 0
model.reactions.EX_pHCA_e.lower_bound = 0

In [37]:
for r in model.reactions:
    if not r.boundary and not 'BIOMASS' in r.id and r.check_mass_balance():
        print(r, r.gene_reaction_rule)
        print(r.check_mass_balance())
        for m in r.metabolites:
            print(m.id, m.formula, m.charge)
        print()

BVMO5: h_c + nadph_c + o2_c + und2one_c --> h2o_c + 0.22 methdca_c + nadp_c + 0.78 nnac_c PP_2805
{'O': 1.3322676295501878e-15, 'C': 1.7763568394002505e-15}
h2o_c H2O 0
h_c H 1
methdca_c C11H22O2 0
nadp_c C21H25N7O17P3 -3
nadph_c C21H26N7O17P3 -4
nnac_c C11H22O2 0
o2_c O2 0
und2one_c C11H22O 0



In [38]:
blocked_rxns = cobra.flux_analysis.find_blocked_reactions(model)

In [39]:
assert len([i for i in blocked_rxns if 'pHCA' in i])==0, 'pHCA branch is blocked'
assert len([i for i in blocked_rxns if 'cinm_Tr' in i])==0, 'cinm branch is blocked'

In [40]:
assert len([i for i in blocked_rxns if '4abz' in i])==0, '4abz branch is blocked'
assert len([i for i in blocked_rxns if '4aPhe' in i])==0, '4aPhe branch is blocked'

In [41]:
for m in model.metabolites:
    if m.compartment != 'c' and m.compartment != 'e' and m.compartment != 'p':
        print(m.id)

## Run FBA
### Establish objective function
Objective function is sum of both biomass and external transamminocinnamate

In [42]:
model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.002467,0,0.00%
cl_e,EX_cl_e,0.002467,0,0.00%
cobalt2_e,EX_cobalt2_e,0.001775,0,0.00%
cu2_e,EX_cu2_e,0.001645,0,0.00%
fe2_e,EX_fe2_e,0.008574,0,0.00%
glc__D_e,EX_glc__D_e,6,6,99.99%
k_e,EX_k_e,0.09254,0,0.00%
mg2_e,EX_mg2_e,0.004112,0,0.00%
mn2_e,EX_mn2_e,0.001645,0,0.00%
mobd_e,EX_mobd_e,0.001905,0,0.00%


Jeremy Zucker says "Better to set a feasible minimal lower bound on biomass (such as 50% max growth) and maximize 4ACA or vice versa. This way biomass flux AND 4ACA fluxes are guaranteed to be nonzero"

In [43]:
model.reactions.BIOMASS_KT2440_WT3.lower_bound = 0.5838113260876195/2

In [44]:
## we expect this output to be non-zero
model.objective = {model.reactions.RgPAL: 1}

sol = model.optimize()
print(sol.objective_value)

1.6047058823529297


$C^{13}$ metabolic flux analysis has found that there is significant flux (~10% of input glucose) through the EDEMP cycle (EDD+EMP) in P. putida ([Nikel et al J Biol Chem 290.43 (2015): 25920-25932](https://www.sciencedirect.com/science/article/pii/S0021925820495730))

In [45]:
def tableFluxes(solution,rxnList):
    "Function to display relevant fluxes"
    values = [solution.fluxes[rxn] for rxn in rxnList]                              # get flux values
    genes = [model.reactions.get_by_id(rxn).gene_reaction_rule for rxn in rxnList]  # get gene names
    df = pd.DataFrame(data= {'Reactions': rxnList, 'Value': values,'Genes': genes}) # Put all of them into a dataframe

    return df

In [46]:
model.objective = model.reactions.RgPAL
solution = model.optimize()

In [47]:
rxnList = ['TKT1','TKT2','EDD','EDA','FBA','FBP','PGI','G6PDH2r','EX_glc__D_e']

In [48]:
tableFluxes(solution,rxnList)

,Reactions,Value,Genes
0,TKT1,0.883297,PP_4965
1,TKT2,-0.844967,PP_4965
2,EDD,4.794643,PP_1010
3,EDA,4.794643,PP_1024
4,FBA,0.000000,PP_4960
5,FBP,0.000000,PP_5040
6,PGI,-0.010154,PP_1808 or PP_4701
7,G6PDH2r,0.000000,PP_1022 or PP_5351 or PP_4042
8,EX_glc__D_e,-6.000000,


We can force flux through the EDEMP cycle to imitate the flux profile found through $^{13}C$ MFA by setting the following bounds:

In [49]:
model.reactions.FBA.upper_bound = -0.5
model.reactions.FBA.lower_bound = -1.2

model.reactions.PGI.upper_bound = -0.5
model.reactions.PGI.lower_bound = -1.2

model.reactions.G6PDH2r.upper_bound = 1.2
model.reactions.G6PDH2r.lower_bound = 0.5

In [50]:
model.objective = model.reactions.RgPAL
solution = model.optimize()
tableFluxes(solution,rxnList)

,Reactions,Value,Genes
0,TKT1,0.867488,PP_4965
1,TKT2,-0.829157,PP_4965
2,EDD,5.310453,PP_1010
3,EDA,5.310453,PP_1024
4,FBA,-0.500000,PP_4960
5,FBP,0.500000,PP_5040
6,PGI,-0.510154,PP_1808 or PP_4701
7,G6PDH2r,0.500000,PP_1022 or PP_5351 or PP_4042
8,EX_glc__D_e,-6.000000,


In [51]:
model.objective = model.reactions.RgPAL
solution = model.optimize()
tableFluxes(solution, ['CHORM', 
                       'R_pheA', 'PHETA1', 'PAL_F', 'R_cinm_Tr', 'EX_cinm_e',
                       'R_tyrA', 'TYRTA', 'PAL_Y', 'R_pHCA_Tr', 'EX_pHCA_e'])

,Reactions,Value,Genes
0,CHORM,0.095506,PP_1769
1,R_pheA,0.000000,PP_1769
2,PHETA1,-0.055765,PP_1972 or PP_3590
3,PAL_F,0.000000,
4,R_cinm_Tr,0.000000,
5,EX_cinm_e,0.000000,
6,R_tyrA,0.000000,PP_1770
7,TYRTA,-0.039741,PP_1972 or PP_3590
8,PAL_Y,0.000000,PP_4692
9,R_pHCA_Tr,0.000000,


EX_pHCA_e & EX_cinm_e have zero flux, yet we know there is pHCA and cinm output from the experimental data. 
Therefore, we will also add more constraints. 

In [52]:
model.reactions.PAL_Y.lower_bound = 1e-5
model.reactions.PAL_F.lower_bound = 1e-5

In [53]:
model.objective = model.reactions.RgPAL
solution = model.optimize()
tableFluxes(solution, ['CHORM', 
                       'R_pheA', 'PHETA1', 'PAL_F', 'R_cinm_Tr', 'EX_cinm_e',
                       'R_tyrA', 'TYRTA', 'PAL_Y', 'R_pHCA_Tr', 'EX_pHCA_e'])

,Reactions,Value,Genes
0,CHORM,0.095526,PP_1769
1,R_pheA,0.000000,PP_1769
2,PHETA1,-0.055775,PP_1972 or PP_3590
3,PAL_F,0.000010,
4,R_cinm_Tr,0.000010,
5,EX_cinm_e,0.000010,
6,R_tyrA,0.000000,PP_1770
7,TYRTA,-0.039751,PP_1972 or PP_3590
8,PAL_Y,0.000010,PP_4692
9,R_pHCA_Tr,0.000010,


In [54]:
solution = model.optimize()
tableFluxes(solution, ['CHORM', 
                       'EX_4abz_e',
                       'EX_4aPhe_e'])

,Reactions,Value,Genes
0,CHORM,0.095526,PP_1769
1,EX_4abz_e,0.000000,
2,EX_4aPhe_e,0.000000,


In [55]:
model.reactions.EX_4abz_e.lower_bound = 1e-5
model.reactions.EX_4aPhe_e.lower_bound = 1e-5

In [56]:
solution = model.optimize()
tableFluxes(solution, ['CHORM', 
                       'EX_4abz_e',
                       'EX_4aPhe_e'])

,Reactions,Value,Genes
0,CHORM,0.095526,PP_1769
1,EX_4abz_e,0.000010,
2,EX_4aPhe_e,0.000010,


In [57]:
# Write model to files
cobra.io.save_json_model(model, '../models/iJN1463_4aca.json')
cobra.io.write_sbml_model(model, '../models/iJN1463_4aca.xml')

Removing 0-flux reactions

In [58]:
# generate a list of reactions where fluxes are 0
zero_flux_rxns = solution.fluxes[solution.fluxes==0].index


In [59]:
# make a model copy
cleaned_model = model.copy()

# remove all zero_flux_rxns from the model
cleaned_model.remove_reactions(zero_flux_rxns, remove_orphans=True)

Read LP format model from file C:\Users\user\AppData\Local\Temp\tmpw358zezu.lp
Reading time = 0.01 seconds
: 2165 rows, 5822 columns, 23044 nonzeros


In [60]:
cleaned_model.optimize()
#sol = model.optimize()
#print(sol.objective_value)

,fluxes,reduced_costs
3HAD160,0.010419,0.000000e+00
1PQQS,0.000065,0.000000e+00
2PQQS,0.000065,0.000000e+00
3HAACOAT140,-0.010667,-0.000000e+00
3HAACOAT141,0.010667,4.440892e-16
...,...,...
R_cinm_Tr,0.000010,0.000000e+00
EX_cinm_e,0.000010,0.000000e+00
PAL_Y,0.000010,-1.953079e+00
R_pHCA_Tr,0.000010,0.000000e+00


In [61]:
# Write model to files
cobra.io.save_json_model(cleaned_model, '../models/iJN1463_JS.json')
cobra.io.write_sbml_model(cleaned_model, '../models/iJN1463_JS.xml')

In [62]:
len(cleaned_model.reactions)

623

## Creating a model with no explicit external species

In [63]:
m1 = cobra.io.load_json_model('../models/iJN1463_JS.json')

In [64]:
new_bd_rxns = []
EX_rxns = []
external_mets = [m for m in m1.metabolites if m.compartment == 'e']
for met in external_mets: 
    for rxn in met.reactions:
        if 'EX' not in rxn.id:
            new_bd_rxns.append(rxn.id)
        else: 
            EX_rxns.append(rxn)

In [65]:
EX_rxns

[<Reaction EX_cl_e at 0x1a5fe52e910>,
 <Reaction EX_co2_e at 0x1a5fe52e9d0>,
 <Reaction EX_ca2_e at 0x1a5fe55ab20>,
 <Reaction EX_cobalt2_e at 0x1a5fe52e700>,
 <Reaction EX_cu2_e at 0x1a5fe52ec70>,
 <Reaction EX_fe2_e at 0x1a5f1c76f40>,
 <Reaction EX_glc__D_e at 0x1a5f1c76d00>,
 <Reaction EX_h_e at 0x1a5f1c76ac0>,
 <Reaction EX_k_e at 0x1a5f1c763a0>,
 <Reaction EX_h2o_e at 0x1a5f1c76be0>,
 <Reaction EX_mg2_e at 0x1a5f1c76730>,
 <Reaction EX_ni2_e at 0x1a5f1c76400>,
 <Reaction EX_mn2_e at 0x1a5f1c765e0>,
 <Reaction EX_mobd_e at 0x1a5f1c768e0>,
 <Reaction EX_o2_e at 0x1a5f1c76190>,
 <Reaction EX_na1_e at 0x1a5f1c76850>,
 <Reaction EX_nh4_e at 0x1a5f1c76790>,
 <Reaction EX_pi_e at 0x1a5f1c760a0>,
 <Reaction EX_so4_e at 0x1a5f1c76100>,
 <Reaction EX_zn2_e at 0x1a5f014de50>,
 <Reaction EX_4aca_e at 0x1a5f1825520>,
 <Reaction EX_4abz_e at 0x1a5f1e40ac0>,
 <Reaction EX_4aPhe_e at 0x1a5f1e400a0>,
 <Reaction EX_cinm_e at 0x1a5f1e400d0>,
 <Reaction EX_pHCA_e at 0x1a5f1d81a00>]

In [66]:
for rxn in EX_rxns: 
    m1.remove_reactions([rxn])

In [67]:
# remove _e metabolites
for met in external_mets: 
    m1.remove_metabolites([met])

# change compartment of _c metabolites to _e 
new_bd_mets = [met.id[:-2] + '_c' for met in external_mets]
for met in new_bd_mets:
    try: # if exists
        m1.metabolites.get_by_id(met).compartment = 'e'
    except: 
        pass

In [68]:
# establish new boundary metabolites
for met in new_bd_mets:
    try: 
        m1.add_boundary(m1.metabolites.get_by_id(met), type="exchange")
    except:
        pass 

In [69]:
# Write model to files
cobra.io.save_json_model(m1, '../models/iJN1463ex_JS.json')
cobra.io.write_sbml_model(m1, '../models/iJN1463ex_JS.xml')

In [70]:
m2 = cobra.io.load_json_model('../models/iJN1463ex_JS.json')
len(m2.reactions)

623